In [169]:
! git pull

Already up to date.


In [170]:
import pandas as pd
import numpy as np

In [171]:
a = np.load("a_list.npy", allow_pickle=True)
o = np.load("o_list.npy", allow_pickle=True)
o.shape

(39673, 125)

In [172]:
obs = np.zeros((o.shape[0],25,5))
act = a.squeeze()
print(obs.shape)
for i, el in enumerate(o):
    # print(el[0].shape, el0)
    
    obs[i,:,:] = el.reshape(25,5)

#removing row with no presence bit
clean_obs = []
for o in obs:
    clean_obs.append(np.round(o[np.where(o[:,0] >0.5) ],2))

(39673, 25, 5)


Actions count

In [173]:
uniq, count = np.unique(act,return_counts=True)
uniq[np.argsort(count)[::-1]][:10], count[np.argsort(count)[::-1]][:10]

array([  558, 13510,    17, 25588], dtype=int64)

In [178]:
clean_obs 


[array([[ 1.  ,  0.92,  0.  ,  0.31,  0.  ],
        [ 1.  ,  0.09,  0.  , -0.03,  0.  ],
        [ 1.  ,  0.2 ,  0.  , -0.03,  0.  ],
        [ 1.  ,  0.3 ,  0.  , -0.03,  0.  ],
        [ 1.  ,  0.4 ,  0.  , -0.03,  0.  ],
        [ 1.  ,  0.51,  0.  , -0.02,  0.  ],
        [ 1.  ,  0.61,  0.5 , -0.05,  0.  ],
        [ 1.  ,  0.71,  0.5 , -0.02,  0.  ],
        [ 1.  ,  0.83,  0.25, -0.03,  0.  ],
        [ 1.  ,  0.92,  0.  , -0.05,  0.  ]]),
 array([[ 1.  ,  1.  ,  0.  ,  0.26,  0.  ],
        [ 1.  ,  0.08,  0.1 , -0.06,  0.05],
        [ 1.  ,  0.19,  0.  , -0.05,  0.  ],
        [ 1.  ,  0.29,  0.17, -0.04,  0.04],
        [ 1.  ,  0.38,  0.  , -0.03,  0.  ],
        [ 1.  ,  0.52,  0.  ,  0.02,  0.  ],
        [ 1.  ,  0.59,  0.65, -0.04,  0.05],
        [ 1.  ,  0.72,  0.5 ,  0.03,  0.  ],
        [ 1.  ,  0.83,  0.25,  0.01,  0.  ],
        [ 1.  ,  0.92,  0.  ,  0.  ,  0.  ]]),
 array([[ 1.  ,  1.  ,  0.  ,  0.25,  0.  ],
        [ 1.  ,  0.05,  0.31, -0.06,  0.07],
      

Lane setup

In [182]:
#lane setup
def find_nearest(array, value):
    lane = np.array([-3,-2,-1,0,1,2,3])
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return lane[idx]

array = np.array([-.75,-.5,-.25,0.0,.25,.5,.75])

for o in clean_obs:
    for o_v in o:
        o_v[2] = find_nearest(array, value=o_v[2])
        


In [183]:
lanes = []
for o in clean_obs:
    for item in np.unique(o[:,2]):
        lanes.append(item)

In [184]:
unique, count = np.unique(np.round(np.array(lanes).flatten(),2), return_counts=True)
unique[np.argsort(count)[::-1]][:10], count[np.argsort(count)[::-1]][:10]

In [190]:
def get_closers(obs):
    closer = dict()
    for lane in np.unique(obs[:,2]):
        closest_idx =np.argmin(obs[np.abs((obs[:,2] - lane))<0.1,1])
        closer[int(lane)] =int(obs[np.abs((obs[:,2] - lane))<0.1,0][closest_idx])
    return closer

In [191]:
get_closers(clean_obs[0])

{0: 2, 1: 9, 2: 7}

In [192]:
def get_ego(obs):
    o = obs[0]
    return o[1:]

In [193]:
def is_faster(ego, closer, const = 0):
    return closer[3] < 0

In [194]:
def is_clean_space(ego, closer, const = 0.2):
    return (closer[1] ) > const 

In [195]:
get_ego(clean_obs[0])

array([0.92, 0.  , 0.31, 0.  ])

In [196]:
get_closers(obs=clean_obs[0])[0]

2

In [197]:
is_faster(get_ego(clean_obs[0]), clean_obs[0][int(get_closers(obs=clean_obs[0])[0])])

True

In [198]:
is_clean_space(get_ego(clean_obs[0]), clean_obs[0][int(get_closers(obs=clean_obs[0])[0])])

False

In [199]:
def create_context(obs):
    key_list = ['ego', 'closers', 'faster', 'clean_space']
    context = {k:[] for k in key_list}
    ego_v = [int(obs_int) for obs_int in get_ego(obs)]
    closers = get_closers(obs[1:])
    for lane,closer_idx in closers.items():
        faster = is_faster(get_ego(obs), obs[int(closer_idx-1)])
        clean = is_clean_space(get_ego(obs), obs[int(closer_idx-1)])
        context['faster'].append((faster,1,int(closer_idx)))
        context['clean_space'].append((clean, int(float(lane))))
        
    context.update({'ego': ego_v, 'closers':[(k,v) for k,v in closers.items()]})
    return context

In [200]:
create_context(clean_obs[0])

{'ego': [0, 0, 0, 0],
 'closers': [(0, 2), (1, 9), (2, 7)],
 'faster': [(True, 1, 2), (True, 1, 9), (True, 1, 7)],
 'clean_space': [(False, 0), (True, 1), (True, 2)]}

In [201]:
def parse_context(obs):
    context = create_context(obs)
    context_string = '{'
    for key, values in context.items():
        if key=="clean_space" or key=="faster" :
            for v in values:
                # print(key)
                if key=="clean_space":
                    if not v[0]:
                        continue
                        context_string += f'not {key}({v[1]}). '
                    else: context_string += f'{key}({v[1]}). '
                else:
                    if not v[0]:
                        continue
                        context_string += f'not {key}{v[1:]}. '
                    else: context_string += f'{key}{v[1:]}. '
                
            #context_string = context_string[:-2]
        elif key == "closers":
                for v in values:
                    context_string += f'{key}{v}. '
                    
                # context_string = context_string[:-2]
        else:
            context_string += f'{key}('
            for v in values:
                context_string += f'{v}, '
            context_string = context_string[:-2]
            context_string += f'). '
    context_string = context_string[:-2]
    context_string += '}'
    return context_string

In [202]:
parse_context(clean_obs[0])

'{ego(0, 0, 0, 0). closers(0, 2). closers(1, 9). closers(2, 7). faster(1, 2). faster(1, 9). faster(1, 7). not clean_space(0). clean_space(1). clean_space(2)}'

In [203]:
def parse_action(action):
    action_map = {0:"lane_left",
                  1:"idle",
                  2:"lane_right",
                  3:"faster",
                  4:"slower",
                  }
    return "{"+ action_map[action]+ "}"

In [204]:
def parse_example(obs, action, iter):
    empty_not = "\{\}"
    return f"#pos({iter}, {parse_action(action)}, {{}}, {parse_context(obs)})\n"

In [205]:
with open("examples.txt", "w") as file:   
    for i, (o, a) in enumerate(zip(clean_obs, act.flatten())):
        # if i ==200:
        # print(o)
        o[:,1:] = o[:,1:] * 100# np.multiply(o,  100)
        o.astype(int)
        # print(o)
        if a == 2:
            file.write(parse_example(o,a,i))
        
file.close()

In [206]:
len(clean_obs), a.shape

(39673, ())

In [207]:
#open a file and write in it parse_example of all element in clean_obs

